In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
from collections import Counter
import autosklearn.classification
import pickle

pd.set_option('max_colwidth', -1)

In [2]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [4]:
data_path = Path('../data')
train = pd.read_csv(Path(data_path, 'normalized_train.csv')).fillna(' ')
test = pd.read_csv(Path(data_path, 'normalized_test.csv')).fillna(' ')

In [5]:
train_text = train['comment_text']
test_text = test['comment_text']

In [6]:
for name in class_names:
    print("{}: {}".format(name, train[train[name] == 1].shape[0]))

toxic: 15294
severe_toxic: 1595
obscene: 8449
threat: 478
insult: 7877
identity_hate: 1405


In [7]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1,1),
    max_features=10000)

train_word_features = word_vectorizer.fit_transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

In [ ]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=20000)

train_char_features = char_vectorizer.fit_transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

In [ ]:
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

In [ ]:
classifiers = {}

In [ ]:
for class_name in class_names:
    print('Fitting {} class'.format(class_name))
    y = class_name
    
    cls = classification.AutoSklearnClassifier(
        time_left_for_this_task=10800,
        per_run_time_limit=1000,
        ml_memory_limit=100000,
        seed=31
        include_estimators=['sgd', 'liblinear_svc', 'libsvm_svc', 'bernoulli_nb', 'multinomial_nb',  'passive_aggressive']
    )
    
    cls.fit(train_features, train[class_name],
        metric=metrics.make_scorer('log_loss', metrics.log_loss, needs_proba=True))
    print(cls.cv_results_)
    print(cls.get_models_with_weights())
    print()
    print('Saving Classifier for {} class'.format(class_name))
    classifiers[class_name] = cls
    print()
    print('Generating Predictions for {} class'.format(class_name))
    submission[class_name] = cls.predict_proba(test_features)[:, 1]
    
submission.to_csv(Path(data_path, 'automl_submission.csv'), index=False)
